# NVIDIA Meeting Recorder Agent

In this notebook, we'll build a meeting recorder agent capable of finding appointments on your Google Calendar, recording the meetings, and sending the recording assets 8video, transcription, and summary) via email.

By the end of the notebook, you'll be able to build and run an agent with xpander that can perform several key tasks, such as:

- Checking your meeting schedule on Google Calendar and retrieving relevant meeting information  
- Creating a meeting recorder to capture the session and monitoring its real-time status  
- Sending the recording, transcription, and summary of the meeting to a specified email address

Along the way, we’ll explore how to persist the agent’s memory using the xpander SDK, and observe the agent’s real-time thought process as it completes each task.

We’ll be using the xpander Python SDK and NVIDIA NIM to bring this agent to life. The xpander SDK manages the workflow and orchestration, while NVIDIA NIM powers the agent's large language model.

## Table of Contents
<ul>
  <li><a href="#Fetch-xpander.ai-API-Key">Fetch xpander.ai API Key</a></li>
  <li><a href="#Fetch-NVIDIA-NIM-API-Key">Fetch NVIDIA NIM API Key</a></li>
  <li><a href="#Setup-and-Build-the-AI-Agent">Setup and Build the AI Agent</a></li>
  <li><a href="#Interact-and-Execute-Agent">Interact and Execute Agent</a>
    <ul>
      <li><a href="#Define-the-Meeting-Agent-Class">Define the Meeting Agent Class</a></li>
      <li><a href="#Initialize-the-Meeting-Agent">Initialize the Meeting Agent</a></li>
      <li><a href="#Example-Use-Case-1:-Test-Agent's-Capabilities">Example Use Case 1: Test Agent's Capabilities</a></li>
      <li><a href="#Example-Use-Case-2:-Retrieve-Upcoming-Calendar-Events">Example Use Case 2: Retrieve Upcoming Calendar Events</a></li>
      <li><a href="#Example-Use-Case-3:-Set-Up-Meeting-Recorder">Example Use Case 3: Set Up Meeting Recorder</a></li>
      <li><a href="#Example-Use-Case-4:-Check-the-Meeting-Recorder-Status">Example Use Case 4: Check the Meeting Recorder Status</a></li>
      <li><a href="#Example-Use-Case-5:-Send-Recording-Assets-via-Email">Example Use Case 5: Send Recording Assets via Email</a></li>
    </ul>
  </li>
  <li><a href="#Summary:-Working-with-the-Meeting-Recorder-Agent">Summary: Working with the Meeting Recorder Agent</a></li>
  <li><a href="#Next-Steps">Next Steps</a></li>
</ul>

Since we're going to use xpander and NVIDIA NIM, then we need to set up two different API keys before we start: **xpander API key** and **NVIDIA NIM API key**. Let's fetch our xpander API key first.

<a id="Fetch-xpander.ai-API-Key"></a>
## Fetch xpander.ai API Key

The simplest way to fetch your xpander API key is by using the xpander CLI.

To install the CLI, you'll need to have Node.js installed on your system. If you haven’t installed Node.js yet, visit their [download page](https://nodejs.org/en/download) and follow the installation instructions.

Once Node.js is installed, run the following command to install the xpander CLI globally:

In [8]:
!npm install -g xpander-cli

⠙⠹⠸^C


Next, log in to your xpander account via the CLI to retrieve your API key by running:

In [ ]:
!xpander login

Your API key will be displayed in the terminal, as shown in the visualization below:

<div style="text-align: center;">
<img src="images/img_1.png" alt="xpander API key display" style="max-height: 300px;">
</div>

Copy your xpander API key and paste it into the `.env` file you created after completing the steps in the **Getting Started** section of this repo.


<a id="Fetch-NVIDIA-NIM-API-Key"></a>
## Fetch NVIDIA NIM API Key

You can get the NVIDIA NIM API key for free. Just head over to [https://build.nvidia.com/models](https://build.nvidia.com/models) and select the LLM you’d like to use.  

For this blueprint, we’ll use NVIDIA’s `llama-3.1-nemotron-ultra-253b-v1`.

<div style="text-align: center;">
    <img src="images/img_2.png" alt="xpander API key display" style="max-height: 300px;">
</div>

Click on **Get API Key** in the top-right corner, then copy your NVIDIA NIM API key and paste it into the `.env` file you created after completing the steps in the **Getting Started** section of this repo.


<a id="Setup-and-Build-the-AI-Agent"></a>
## Set Up and Build the AI Agent

Now that we've set up the required credentials, we can start building our meeting recorder agent.

We've prepared a template that you can use to instantly spin up the agent. Just click the button below to build the agent using our template.


<br>
<br>
<div style="text-align: center;">
    <a href="https://app.xpander.ai/templates/383f7607-0223-42a8-a472-5c7b1fdf6315" target="_blank">
        <button style="
            background-color: #6741d9;
            color:white;
            padding: 10px 20px;
            font-size: 14px;
            border: none;
            border-radius: 8px;
            cursor: pointer;
    ">
            Build AI Meeting Recorder Agent with xpander Template
        </button>
    </a>
</div>
<br>
<br>


After clicking the button, you'll be redirected to the xpander console, as shown in the visualization below:

<div style="text-align: center;">
    <img src="images/img_3.png" alt="" style="max-height: 300px;">
</div>

Using a template to build an AI agent is straightforward. The template handles everything needed to set up the agent:

- It configures the required interface(s), such as Google Calendar, along with the relevant operations (e.g., retrieve events, create events, delete events, etc.).
- It orchestrates the agent’s logical workflow, if necessary.

<div style="text-align: center;">
    <img src="images/img_4.png" alt="" style="max-height: 300px;">
</div>

Once you click **Finish**, you’ll see that a new AI agent called **AI Meeting Recorder Agent (Imported)** has been successfully created in your xpander account.

<div style="text-align: center;">
    <img src="images/img_5.png" alt="" style="max-height: 300px;">
</div>

Clicking on the newly created agent will show you the agent's orchestration derived from the template, along with the configured interfaces needed to run the agent.

Since a key feature of this agent is the ability to check your Google Calendar for scheduled meetings, you’ll need to grant xpander permission to access your calendar.  

If you haven’t granted this permission yet, you’ll see a warning icon next to the Google Calendar operation, as shown in the visualization below:

<div style="text-align: center;">
    <img src="images/img_6.png" alt="" style="max-height: 300px;">
</div>

To grant xpander permission to access your Google Calendar, click on the warning icon. Then, click **Sign In with Google Calendar** and follow the prompts.

Once you've granted permission to xpander, the warning icon should disappear.

Next, we need to bind the agent ID with our API key so we can interact with the agent. We’ll also need to store the agent ID in the `.env` file. To do this, click on the **SDK** node at the top of the graph visualization in your xpander console, as shown below:

<div style="text-align: center;">
    <img src="images/img_7.png" alt="" style="max-height: 300px;">
</div>

Copy the agent ID into the `.env` file. Then, click **Assign API Keys** and tick the box next to your xpander API key, as shown below:

<div style="text-align: center;">
    <img src="images/img_8.png" alt="" style="max-height: 300px;">
</div>

And that's it. We're now ready to interact with our agent!


<a id="Interact-and-Execute-Agent"></a>

## Interact and Execute Agent

In this section, we’ll execute and interact with the agent we created using the template. As mentioned in the introduction of this notebook, we’ll use the agent to perform several tasks, including:

- Checking our meeting schedule on Google Calendar and retrieving relevant meeting information  
- Creating a meeting recorder to capture the session and monitoring its real-time status  
- Sending the recording, transcription, and summary of the meeting to a specified email address  

First, let’s retrieve all the necessary credentials: the xpander API key, the NVIDIA NIM API key, and the agent ID we created in the above steps.

In [1]:
import os
import datetime
import time
import re
import ast
import random
from dotenv import load_dotenv
from xpander_sdk import XpanderClient, LLMProvider, ToolCall, ToolCallType, Tokens, LLMTokens, Agent
from openai import OpenAI

# Load environment variables
load_dotenv()

# Verify required API keys are available
xpander_api_key = os.environ.get("XPANDER_API_KEY")
agent_id = os.environ.get("XPANDER_AGENT_ID")
nvidia_nim_api_key = os.environ.get("NVIDIA_NIM_API_KEY")


if not all([xpander_api_key, nvidia_nim_api_key, agent_id]):
    missing = []
    if not xpander_api_key: missing.append("XPANDER_API_KEY")
    if not nvidia_nim_api_key: missing.append("NVIDIA_NIM_API_KEY")
    if not agent_id: missing.append("XPANDER_AGENT_ID")
    print(f"❗️ Missing required environment variables: {', '.join(missing)}")
else:
    print(f"✅ All required API keys loaded successfully")
    
print(f"Open the agent alogside this notebook at https://app.xpander.ai/agents/{agent_id}")

✅ All required API keys loaded successfully
Open the agent alogside this notebook at https://app.xpander.ai/agents/d32c6a03-775a-4edc-a754-0e1ddc5b7b75


<a id="Define-the-Meeting-Agent-Class"></a>
### Define the Meeting Agent Class

Now, let’s create our `MeetingAgent` class. This class will handle both the initialization of the agent and the logic for processing input prompts and generating responses.

For each prompt the agent receives, it processes the input and generates a response through a looped operation. Each iteration of the loop consists of the following steps:

- The LLM generates a response based on the current state.

- It evaluates whether it can complete the task directly or if it needs to call a tool (e.g., Google Calendar, xpander built-in actions, etc.).

- If a tool is needed, the agent selects the appropriate tool and provides the required input arguments.

- xpander executes the tool call and returns the result to the agent for the next iteration.

- Once the LLM determines that it has all the necessary information to answer the user’s query, it calls the `xpfinish-agent-execution-finished` function, essentially ending the loop.

In [2]:
class MeetingAgent:
    """Class for running the meeting recorder agent"""
    
    def __init__(self, nvidia_nim_api_key, xpander_api_key, agent_id):
        self.agent_id = agent_id
        self.xpander_client = XpanderClient(api_key=xpander_api_key)
        self.openai_client =  OpenAI(
            base_url = "https://integrate.api.nvidia.com/v1",
            api_key = nvidia_nim_api_key
            )
 
        self.agent : Agent = self.xpander_client.agents.get(agent_id=self.agent_id)


    def run(self, prompt=None, thread_id=None):
        """Run the agent with the given prompt"""

        task = prompt or f"Please check the status of all recorded meetings."

        print("-" * 60)
        print(f"{'Continuing conversation in thread: ' + thread_id if thread_id else 'Starting a new conversation thread'}")
        print(f"Processing Task: \n\n{task}\n\n")
        print("-" * 60)
           
        # Create task with or without thread_id
        self.agent.add_task(input=task, thread_id=thread_id if thread_id else None)
        self.agent.add_messages([{"role": "assistant", "content": "the time now is : " + datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}])

        # Initialize token tracking and timing
        execution_tokens = Tokens(worker=LLMTokens(completion_tokens=0, prompt_tokens=0, total_tokens=0))
        execution_start_time = time.perf_counter()
          
        # Run the agent until it's finished
        while not self.agent.is_finished():

            # Track start time for this inference
            start_time = time.perf_counter()

            # Get response from OpenAI and process
            response = self.openai_client.chat.completions.create(
                model="nvidia/llama-3.1-nemotron-ultra-253b-v1",
                messages= self.agent.messages,
                temperature=0.0,
                tools=self.agent.get_tools(),
                tool_choice=self.agent.tool_choice

            )

            # Track token usage
            execution_tokens.worker.completion_tokens += response.usage.completion_tokens
            execution_tokens.worker.prompt_tokens += response.usage.prompt_tokens
            execution_tokens.worker.total_tokens += response.usage.total_tokens

                
            # Report LLM usage to Xpander
            self.agent.report_llm_usage(
                llm_response=response.model_dump(),
                llm_inference_duration=time.perf_counter() - start_time,
                llm_provider=LLMProvider.NVIDIA_NIM
            )
            self.agent.add_messages(response.model_dump())

            tool_calls = self.agent.extract_tool_calls(llm_response=response.model_dump())
            responses = self.agent.run_tools(tool_calls=tool_calls)
            
            for response in responses:
                print("Tool call function name : ", response.function_name)
                print("HTTP Status code : ", response.status_code) 
        # Process results
        execution_end_time = time.perf_counter()
        result = self.agent.retrieve_execution_result()
        print(f"Your thread ID is: {result.memory_thread_id}")
        result_text = result.result
    
        # Report execution metrics to Xpander
        self.agent.report_execution_metrics(
            llm_tokens=execution_tokens,
            ai_model="nvidia/llama-3.1-nemotron-ultra-253b-v1",
        )
        print(f"Status: {result.status}")
        print(f"Result: {result_text}")
        print(f"Execution duration: {execution_end_time - execution_start_time:.2f} seconds")
    
        # Return both the result text and thread ID
        return result_text, result.memory_thread_id

<a id="Initialize-the-Meeting-Agent"></a>
### Initialize the Meeting Agent

Now that we’ve defined the logic for how the agent should process user prompts, let’s go ahead and initialize our meeting agent using the `MeetingAgent` class we just created.

We also need to provide the agent with clear instructions, roles, and specific goals. This ensures that:

- It understands its task and objective

- It follows the correct workflow when handling user requests

- It knows which tools to use in various situations to effectively complete a task

In [3]:
# Initialize the meeting agent
meeting_agent = MeetingAgent(
    nvidia_nim_api_key=nvidia_nim_api_key,
    xpander_api_key=xpander_api_key,
    agent_id=agent_id
)

meeting_agent.agent.instructions.general = f"""
Reasoning mode: ON
You are an expert in composing functions. You are given a question and a set of possible functions.
Based on the question, you will need to make one or more function/tool calls to achieve the purpose.
To list user's calendar, use CalendarEventManagementGetCalendarEventsById
If none of the function can be used, point it out. If the given question lacks the parameters required by the function,
also point it out. You should only return the function call in tools call sections.

If you decide to invoke any of the function(s), you MUST put it in the format of <TOOLCALL>[func_name1{{'params_name1': 'params_value1', 'params_name2': 'params_value2'...}}]</TOOLCALL>

You SHOULD NOT include any other text in the response.
Here is a list of functions in JSON format that you can invoke. 

<AVAILABLE_TOOLS>{meeting_agent.agent.get_tools(llm_provider=LLMProvider.NVIDIA_NIM)}</AVAILABLE_TOOLS>
"""

meeting_agent.agent.instructions.role = """
You are a helpful meeting recorder AI agent with access to Google Calendar and Meeting Recording tools.
"""

meeting_agent.agent.instructions.goal = """
Your goal is to help users answer their query, record their meetings, check the status of the recording tools, and get the assets of the recorded meeting using the recording bot id. However, please don't check the status of recorder unless explicitly requested by the user.
"""

b'{"stdout":"bG9hZGluZyBhZ2VudCBkMzJjNmEwMy03NzVhLTRlZGMtYTc1NC0wZTFkZGM1YjdiNzUK"}\n'
b'{"stdout":"cnVubmluZyB0b29sIHhwZmluaXNoLWFnZW50LWV4ZWN1dGlvbi1maW5pc2hlZCBvbiBhZ2VudCBkMzJjNmEwMy03NzVhLTRlZGMtYTc1NC0wZTFkZGM1YjdiNzUgd2l0aCBleGVjdXRpb24gMzYxMzVhMmMtNjM5Mi00MDVlLWE5YmItZWRlZmYyZDQyN2M5Cg=="}\n'
b'{"stdout":"cnVubmluZyB0b29sIENhbGVuZGFyRXZlbnRNYW5hZ2VtZW50R2V0Q2FsZW5kYXJFdmVudHNCeUlkIG9uIGFnZW50IGQzMmM2YTAzLTc3NWEtNGVkYy1hNzU0LTBlMWRkYzViN2I3NSB3aXRoIGV4ZWN1dGlvbiAyMmNhZDg0Ni00NTE0LTRmMTgtYTMzNC0xNWYwNGZmZGYzZjEK"}\n'
b'{"stdout":"cnVubmluZyB0b29sIHhwZmluaXNoLWFnZW50LWV4ZWN1dGlvbi1maW5pc2hlZCBvbiBhZ2VudCBkMzJjNmEwMy03NzVhLTRlZGMtYTc1NC0wZTFkZGM1YjdiNzUgd2l0aCBleGVjdXRpb24gMjJjYWQ4NDYtNDUxNC00ZjE4LWEzMzQtMTVmMDRmZmRmM2YxCg=="}\n'
b'{"stdout":"cnVubmluZyB0b29sIFhwYW5kZXJNZWV0aW5nTWFuYWdlbWVudENyZWF0ZVJlY29yZGluZ0JvdCBvbiBhZ2VudCBkMzJjNmEwMy03NzVhLTRlZGMtYTc1NC0wZTFkZGM1YjdiNzUgd2l0aCBleGVjdXRpb24gOTc0Mjk4MDgtOGVlYS00Njc0LWFmMWUtNmY4NTk5MDNhNjk0Cg=="}\n'
b'{"stdout":"cnVubmluZyB0b29s

<a id="Example-Use-Case-1:-Test-Agent's-Capabilities"></a>
### Example Use Case 1: Test Agent's Capabilities

Once we have setup everything, let's test the agent capabilities by using the following prompt:

In [4]:
prompt = """
Hi! What can you do?
"""

result, thread_id = meeting_agent.run(prompt=prompt)

------------------------------------------------------------
Starting a new conversation thread
Processing Task: 


Hi! What can you do?



------------------------------------------------------------
Tool call function name :  xpfinish-agent-execution-finished
HTTP Status code :  200
Your thread ID is: 527a3ae8-6c47-40f1-9937-63ad7262aa33
Status: ExecutionStatus.COMPLETED
Result: I can help you with your Google Calendar events, record meetings, check the status of recording tools, and retrieve meeting assets using the recording bot ID. Please let me know how I can assist you today!
Execution duration: 18.67 seconds


After completing the task, xpander generates a thread ID for the agent's run. This thread ID is crucial for maintaining conversation history. It allows the agent to understand context and handle follow-up questions related to a previous query. We can simply include the same thread ID in subsequent prompts.

<a id="Example-Use-Case-2:-Retrieve-Upcoming-Calendar-Events"></a>
### Example Use Case 2: Retrieve Upcoming Calendar Events

Since our agent now has access to Google Calendar, we can ask it to retrieve upcoming meetings from our schedule. To execute the agent with memory persistence, use the thread ID retrieved from the previous interaction and pass it as an argument along with your prompt.

In [5]:
# Use the same thread_id to maintain context from the authorization
prompt = """
List my upcoming meetings on the 15th of May 2025 and 3 days afterwards.
For each meeting, include the title, date, time, and participants.
"""

meeting_agent.run(prompt=prompt, thread_id=thread_id)

------------------------------------------------------------
Continuing conversation in thread: 527a3ae8-6c47-40f1-9937-63ad7262aa33
Processing Task: 


List my upcoming meetings on the 15th of May 2025 and 3 days afterwards.
For each meeting, include the title, date, time, and participants.



------------------------------------------------------------
Tool call function name :  CalendarEventManagementGetCalendarEventsById
HTTP Status code :  200
Tool call function name :  xpfinish-agent-execution-finished
HTTP Status code :  200
Your thread ID is: 527a3ae8-6c47-40f1-9937-63ad7262aa33
Status: ExecutionStatus.COMPLETED
Result: Here are your upcoming meetings on May 15th, 2025 and the next 3 days:

1. **Onboarding Employee**
   - Date: May 15, 2025
   - Time: 5:15 AM - 5:16 AM UTC
   - Participants: christofbenbach@yahoo.com, marcellusruben@gmail.com

Please note that the other events listed were cancelled.
Execution duration: 16.55 seconds


('Here are your upcoming meetings on May 15th, 2025 and the next 3 days:\n\n1. **Onboarding Employee**\n   - Date: May 15, 2025\n   - Time: 5:15 AM - 5:16 AM UTC\n   - Participants: christofbenbach@yahoo.com, marcellusruben@gmail.com\n\nPlease note that the other events listed were cancelled.',
 '527a3ae8-6c47-40f1-9937-63ad7262aa33')

As you can see, the agent called `CalendarEventManagementGetCalendarEventsById` to retrieve the ID of the relevant event from our Google Calendar. Once it had all the necessary information, it called `xpfinish-agent-execution-finished` to signal that it was ready to generate a response for the user.

<a id="Example-Use-Case-3:-Set-Up-Meeting-Recorder"></a>
### Example Use Case 3: Set Up Meeting Recorder

We can also ask our agent to set up a recording bot for a particular meeting with prompt similar to this:

In [6]:
# Create a meeting recorder for a specific meeting
prompt = """
Can you create a recorder for the onboarding meeting? 
"""

meeting_agent.run(prompt=prompt, thread_id=thread_id)

------------------------------------------------------------
Continuing conversation in thread: 527a3ae8-6c47-40f1-9937-63ad7262aa33
Processing Task: 


Can you create a recorder for the onboarding meeting? 



------------------------------------------------------------
Tool call function name :  XpanderMeetingManagementCreateRecordingBot
HTTP Status code :  200
Tool call function name :  XpanderMeetingManagementCheckRecorderStatus
HTTP Status code :  200
Tool call function name :  xpfinish-agent-execution-finished
HTTP Status code :  200
Your thread ID is: 527a3ae8-6c47-40f1-9937-63ad7262aa33
Status: ExecutionStatus.COMPLETED
Result: The recorder bot for the onboarding meeting has been created with ID 764bb4f6-1fa1-4659-a7e2-1e8d4566c74a. However, the current status is unknown, and no assets are available yet. Please check the status again later.
Execution duration: 27.54 seconds


('The recorder bot for the onboarding meeting has been created with ID 764bb4f6-1fa1-4659-a7e2-1e8d4566c74a. However, the current status is unknown, and no assets are available yet. Please check the status again later.',
 '527a3ae8-6c47-40f1-9937-63ad7262aa33')

The agent called the `XpanderMeetingManagementCreateRecordingBot` tool to create and set up a recorder for the appropriate meeting. It first fetched the meeting link, then used that link as input when initializing the recording bot. 

<a id="Example-Use-Case-4:-Check-the-Meeting-Recorder-Status"></a>
### Example Use Case 4: Check the Meeting Recorder Status

This time, we’ll ask the agent for the status of the meeting recorder bot it created in the previous interaction. To do this, we’ll use the same thread ID from earlier runs to maintain the agent’s memory and context.

In [7]:
# Check the status of our meeting recorder
prompt = """
Check the status of the meeting recorder I just created. Get me the link of the meeting assets if it has finished recording.
"""

meeting_agent.run(prompt=prompt, thread_id=thread_id)

------------------------------------------------------------
Continuing conversation in thread: 527a3ae8-6c47-40f1-9937-63ad7262aa33
Processing Task: 


Check the status of the meeting recorder I just created. Get me the link of the meeting assets if it has finished recording.



------------------------------------------------------------
Tool call function name :  XpanderMeetingManagementCheckRecorderStatus
HTTP Status code :  200
Tool call function name :  xpfinish-agent-execution-finished
HTTP Status code :  200
Your thread ID is: 527a3ae8-6c47-40f1-9937-63ad7262aa33
Status: ExecutionStatus.COMPLETED
Result: The meeting recording is done. Here are the assets:

Video: https://links.xpander.ai/wzfnysq (mp4)
Transcript: https://links.xpander.ai/j2jij9n (txt)
Execution duration: 23.06 seconds


('The meeting recording is done. Here are the assets:\n\nVideo: https://links.xpander.ai/wzfnysq (mp4)\nTranscript: https://links.xpander.ai/j2jij9n (txt)',
 '527a3ae8-6c47-40f1-9937-63ad7262aa33')

To complete the task, the agent called the `XpanderMeetingManagementCheckRecorderStatus` function to retrieve the current status of the recorder bot. Note that to use this function, the agent must provide the recorder bot ID, which it retrieved during the previous interaction.

In the example above, since the recorder bot has finished recording the meeting, the recording assets, such as the video and transcript, are now available for download.



<a id="Example-Use-Case-5:-Send-Recording-Assets-via-Email"></a>
### Example Use Case 5: Send Recording Assets via Email

In the last interaction, the `XpanderMeetingManagementCheckRecorderStatus` tool returns meeting assets such as the meeting video and transcript after the recorder bot has finished recording. We can now ask the agent to give us a short summary of the meeting over email and attach the meeting assets (video and transcript) in the email.

In [8]:
prompt = """
Please send the link of the meeting assets (video and transcript) to this email address: marcellusruben@gmail.com. 
Also, write a short key points of the meeting in the <email body> part of the following email template:

Subject: Meeting Assets

Dear <Recipient>,

<email body>

Here are the links to your assets:
- Video: <video link>
- Transcript: <transcript link> 
"""

meeting_agent.run(prompt=prompt, thread_id=thread_id)

------------------------------------------------------------
Continuing conversation in thread: 527a3ae8-6c47-40f1-9937-63ad7262aa33
Processing Task: 


Please send the link of the meeting assets (video and transcript) to this email address: marcellusruben@gmail.com. 
Also, write a short key points of the meeting in the <email body> part of the following email template:

Subject: Meeting Assets

Dear <Recipient>,

<email body>

Here are the links to your assets:
- Video: <video link>
- Transcript: <transcript link> 



------------------------------------------------------------
Tool call function name :  XpanderMessagingServiceSendEmailWithContent
HTTP Status code :  200
Tool call function name :  xpfinish-agent-execution-finished
HTTP Status code :  200
Your thread ID is: 527a3ae8-6c47-40f1-9937-63ad7262aa33
Status: ExecutionStatus.COMPLETED
Result: I have finished my job successfully. The email with the meeting assets and key points has been sent to marcellusruben@gmail.com.
Executi

('I have finished my job successfully. The email with the meeting assets and key points has been sent to marcellusruben@gmail.com.',
 '527a3ae8-6c47-40f1-9937-63ad7262aa33')

As you can see, our agent called the `XpanderMessagingServiceSendEmailWithContent` function to complete the task. If you head over to the email that you specified in the prompt, you'll receive an email from the agent containing the link to the video recording as well as the transcript and the summary of the meeting.

You can also inspect the workflow of your agent from xpander Workbench by clicking on the **Activity** view at the top of the graph visualization of your xpander console.

![img_9](images/img_9.png)

There you can inspect several information via two key components:

- **Thread**: A thread is created every time you assign a task or input prompt to an agent. It includes information such as the total number of tokens generated and the creation timestamp.

- **Log**: This contains a detailed breakdown of your agent's execution steps. You'll see the agent's responses, any tools it called, the payload used for those tool calls, and the responses received after each call.


<a id="Summary:-Working-with-the-Meeting-Recorder-Agent"></a>
## Summary: Working with the Meeting Recorder Agent

In this notebook, we've explored how to use the AI Meeting Recorder Agent for various meeting-related tasks:

1. **Google Calendar Integration**:
   - Authorization and authentication
   - Retrieving upcoming meetings

2. **Meeting Recording Capabilities**:
   - Setting up recording bots for meetings
   - Checking recording status
   - Processing completed recordings

3. **Email Notifications**:
   - Sending meeting summaries via email
   - Including action items and key points

4. **Memory and Context Preservation**:
   - Using thread IDs to maintain conversation context

<a id="Next-Steps"></a>
## Next Steps

In this blueprint, we've created a meeting recorder agent with the help of xpander.ai and NVIDIA NIM. You can now use this blueprint as a starting point to incorporate NVIDIA AI into xpander workflow, or apply it to different use cases that combine xpander and NVIDIA NIM.


We can also further enhance the use of our Meeting Recorder Agent, as examples:

1. Customize the agent with additional tools or capabilities
2. Integrate with other calendar systems or communication platforms
3. Develop more sophisticated workflows for specific meeting types
4. Implement authentication for multiple users

Visit the [Xpander documentation](https://docs.xpander.ai) for more information on extending your agent's capabilities.